In [ ]:
%matplotlib notebook

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import requests

pd.set_option('display.max_rows', 500)

In [13]:
covid_df = pd.read_csv(os.path.join("COVID","03-16-2020.csv"))
world_pop_df = pd.read_csv(os.path.join('GDP', 'world_countries_gdp.csv'))
median_age_df = pd.read_csv(os.path.join('Median Age', 'output','median_age.csv'))
life_expectancy = pd.read_csv(os.path.join('Life Expectancy','output','life_expectancy.csv'))
unemployment_rate = pd.read_csv(os.path.join('Unemployment Rate','output','unemployment_rate.csv'))
medical_expenditure = pd.read_csv(os.path.join("Medical Expenditure", "NHA indicators.csv"),encoding = 'ISO-8859-1')
poverty = pd.read_csv(os.path.join('Poverty','output','poverty.csv'))
literacy = pd.read_csv(os.path.join('Literacy_Rate','literacy_rate.csv'))
bed_density = pd.read_csv(os.path.join('Hospital_beds','beds_density.csv'))
gender_ratio = pd.read_csv(os.path.join('Gender Ratio','output','gender_ratio.csv'))

#number of countries in each csv

print(f"COVID-19 Countries: {covid_df['Country/Region'].nunique()}")
print(f"World Population Countries: {world_pop_df['country'].nunique()}")
print(f"Median Age Countries: {median_age_df['Country'].nunique()}")
print(f"Life Expectancy Countries: {life_expectancy['Country'].nunique()}")
print(f"Unemployment Rate Countries: {unemployment_rate['Country'].nunique()}")
print(f"Medical Expenditure Countries: {medical_expenditure['Countries'].nunique()}")
print(f"Poverty Countries: {poverty['Country'].nunique()}")
print(f"Literacy Countries: {literacy['Country'].nunique()}")
print(f"Bed Density Countries: {bed_density['Country'].nunique()}")
print(f"Gender Ratio Countries: {gender_ratio['Country'].nunique()}")


COVID-19 Countries: 156
World Population Countries: 211
Median Age Countries: 259
Life Expectancy Countries: 259
Unemployment Rate Countries: 259
Medical Expenditure Countries: 188
Poverty Countries: 259
Literacy Countries: 259
Bed Density Countries: 259
Gender Ratio Countries: 259


In [4]:
#get date only
#map takes in a function. So we use a temporary function (lambda)
covid_df['Last Update'] = covid_df['Last Update'].map(lambda x: x[0:10])

In [5]:
#clean data
grouped_covid_df = covid_df.groupby(['Country/Region']).sum()
grouped_covid_df.drop(['Latitude','Longitude'],1,inplace=True)
grouped_covid_df.rename(index = {"US":"United States","Taiwan*":"Taiwan"}, inplace=True)
grouped_covid_df.reset_index(inplace=True)
grouped_covid_df.rename(columns = {"Country/Region":"Country"}, inplace=True)
medical_expenditure.columns = ['Countries', '2017 Health Budget per Capita ($)']
#grouped_covid_df.query('Country_Region.str.contains("Congo")',engine='python',inplace=True)

In [6]:
#clean data
grouped_world_pop_df = world_pop_df.groupby('country').sum()
grouped_world_pop_df.drop(['rank','unGDP'],1,inplace=True)
grouped_world_pop_df.rename(index = {"South Korea":"Korea, South","Bahamas":"The Bahamas"}, 
                            columns = {"imfGDP": "GDP","gdpPerCapita": "GDP Per Capita","pop":"Population"},
                            inplace=True)
grouped_world_pop_df.index.name = 'Country'
grouped_world_pop_df.reset_index(inplace=True)

In [7]:
clean_median_age_df = median_age_df.copy()
#replace country column with string replace
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='And',repl='and',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='The',repl='the',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Congo, Democratic Republic Of the',repl='Republic of the Congo',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='andorra',repl='Andorra',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Bahamas, the',repl='The Bahamas',case=False)
clean_median_age_df.sort_values(by='Country',ascending=True,inplace=True)
clean_median_age_df.reset_index(drop=True,inplace=True)

In [ ]:
merged_df = grouped_covid_df.merge(grouped_world_pop_df,how='outer',on='Country')
#merged_df.dropna(inplace=True)
merged_df.reset_index(drop=True,inplace=True)
#merged_df['GDP Per Capita'] = merged_df['GDP Per Capita'].map('{:.0f}'.format)
#merged_df['Population'] = (merged_df['Population'] * 1000).astype('int64')

In [ ]:
complete_df = merged_df.merge(clean_median_age_df,how='outer',on='Country')
complete_df.sort_values(by='Country',ascending=True,inplace=True)
complete_df.reset_index(drop=True,inplace=True)
#complete_df.dropna(inplace=True)
#complete_df.reset_index(drop=True,inplace=True)

complete_df

In [ ]:
complete_df.to_csv('Final Data\complete_data.csv', index=False)

In [7]:
target_url = "https://raw.githubusercontent.com/iancoleman/cia_world_factbook_api/master/data/factbook.json"
cia = requests.get(target_url).json()

In [9]:
beds = {}

for country in cia['countries'].keys():
    try:
        beds[cia['countries'][country]['data']['name']] = cia['countries'][country]['data']['people']['hospital_bed_density']['beds_per_1000_population']
    except Exception as ex:
        #print(f"{cia['countries'][country]['data']['name']} does not have data")
        #print(ex)   
        beds[cia['countries'][country]['data']['name']] = ''

Beds_df = pd.DataFrame(beds.items(), columns={'Country','Hospital_Beds_per_1000'})
Beds_df.set_index = ('Country')
Beds_df

,Country,Hospital_Beds_per_1000
0,World,
1,Afghanistan,0.5
2,Akrotiri,
3,Albania,2.9
4,Algeria,1.9
5,American Samoa,
6,Andorra,2.5
7,Angola,
8,Anguilla,
9,Antarctica,


In [17]:
literacy = {}

for country in cia['countries'].keys():
    try:
        literacy[cia['countries'][country]['data']['name']] = cia['countries'][country]['data']['people']['literacy']['total_population']['value']
    except Exception as ex:
        literacy[cia['countries'][country]['data']['name']] = ''

literacy_df = pd.DataFrame(literacy.items(), columns={'Country',"Literacy_Rate"})
literacy_df.set_index = ('Country')
literacy_df

,Country,Literacy_Rate
0,World,86.2
1,Afghanistan,38.2
2,Akrotiri,
3,Albania,97.6
4,Algeria,80.2
5,American Samoa,
6,Andorra,100
7,Angola,71.1
8,Anguilla,
9,Antarctica,


In [12]:
Beds_df.to_csv('beds_density.csv', index=False)

In [15]:
literacy_df.to_csv('literacy_rate.csv', index=False)